In [ ]:
import pandas as pd
import numpy as 

In [2]:
df = pd.read_csv('./csv/all_exercises_Química.csv')

In [3]:
df.head()

,subject,year,topic,exam,exercise,statement
0,Química,2000,Equilibrio Químico,Junio,"Ejercicio 3, Opción B","En la tabla adjunta se recogen los valores, a ..."
1,Química,2000,Equilibrio Químico,Reserva 1,"Ejercicio 5, Opción A","A 613* K, el valor de K, para la reacción: Fe,..."
2,Química,2000,Equilibrio Químico,Reserva 1,"Ejercicio 3, Opción B",Suponga el siguiente sistema en equilibrio: UO...
3,Química,2000,Equilibrio Químico,Reserva 1,"Ejercicio 4, Opción B",a) Dibuje el diagrama de entalpía teniendo en ...
4,Química,2000,Equilibrio Químico,Reserva 2,"Ejercicio 3, Opción B","Indique, razonadamente, si las siguientes afir..."


In [4]:
df.topic.unique()

array(['Equilibrio Químico', 'Cantidad Química', 'Ácido Base',
       'Conf. Electrónica', 'Termoquímica', 'Reactividad Orgánica',
       'Enlace Químico', 'Formulación', 'Reacciones Redox', 'Solubilidad'],
      dtype=object)

In [5]:
keywords = {
    'Equilibrio Químico': None,
    'Cantidad Química': None,
    'Ácido Base': None,
    'Conf. Electrónica': None,
    'Termoquímica': None,
    'Reactividad Orgánica': None,
    'Enlace Químico': None,
    'Formulación': None,
    
    'Reacciones Redox': {
        'iónica y molecular' : 'Ajuste reacción redox',
        "E*" : 'Pilas galvánicas',
        "E'" : 'Pilas galvánicas',
        'F = 96500 C' : 'Ley de Faraday'
    },

    'Solubilidad': {
        'verdader' : 'Verdadero o Falso',
        'veracidad' : 'Verdadero o Falso',
        'en agua' : 'Efecto ion común problema',
        'empiece a precipitar' : 'Cantidad a añadir para que precipite',
        'precipitado' : '¿Se formará precipitado?',
        ' M ': 'Efecto ion común problema',
        'pH' : 'Efecto del pH en la solubilidad',
    },

    'Termoquímica' : {
	'verdader' : 'Verdadero o Falso',
	'C-C': 'Entalpías de enlace',
	'S*' : 'Cálculo de entropía',
	"S'":  'Cálculo de entropía',
	'[' : 'Entalpías de formación',
	'a partir de los siguientes datos' : 'Ley de Hess',
	'a volumen constante' : 'Relación entalpía y energía interna',
	'trabajo' : '1er Principio de la Termodinámica',
	'interna' : '1er Principio de la Termodinámica',
	'entalpías de combustión' : 'Ley de Hess',
	'temperatura' : 'Energía libre de Gibbs teórico',
	}



}


In [6]:
topics = df.topic.unique()

for i in topics:
    exercise_types = []
    keywords_topic = keywords[i]
    # there are established keywords
    # for the current topic
    if keywords_topic:
        subset = df[df.topic == i]
        for statement in subset.statement:
            classified = False
            for k, v in keywords.items():
                if k in statement:
                    exercise_types.append(v)
                    classified = True
                    break
            if not classified:
                exercise_types.append('other')
        df.loc[df.topic == i, 'exercise_type'] = exercise_types

In [21]:
df.exercise_type = df.exercise_type.replace('nan', pd.NA)

In [22]:
df[~df.exercise_type.isna()]

,subject,year,topic,exam,exercise,statement,exercise_type
762,Química,2000,Termoquímica,Reserva 1,"Ejercicio 4, Opción B",a) Dibuje el diagrama de entalpía teniendo en ...,other
763,Química,2000,Termoquímica,Reserva 2,"Ejercicio 6, Opción A","El amoniaco, a 25 “C y 1 atm, se puede oxidar ...",other
764,Química,2000,Termoquímica,Reserva 3,"Ejercicio 5, Opción B",a) Calcule la variación de entalpía estándar c...,other
765,Química,2000,Termoquímica,Reserva 4,"Ejercicio 5, Opción A",a) Calcule la variación de entalpía estándar d...,other
766,Química,2000,Termoquímica,Septiembre,"Ejercicio 3, Opción B",Indique razonadamente si las siguientes afirma...,other
...,...,...,...,...,...,...,...
1677,Química,2023,Solubilidad,Reserva 3,Ejercicio B5,Razone si las siguientes afirmaciones son verd...,other
1678,Química,2023,Solubilidad,Reserva 3,Ejercicio C2,"A una temperatura determinada, el producto de ...",other
1679,Química,2023,Solubilidad,Reserva 4,Ejercicio C2,"A una temperatura determinada, la solubilidad ...",other
1680,Química,2023,Solubilidad,Julio,Ejercicio C2,Basándose en las reacciones químicas correspon...,other


In [8]:
len(exercise_types)

87

In [9]:
subset['exercise_type'] = exercise_types

/tmp/ipykernel_6098/3989028331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['exercise_type'] = exercise_types


In [10]:
subset.sample()

,subject,year,topic,exam,exercise,statement,exercise_type
1628,Química,2015,Solubilidad,Reserva 2,"Ejercicio 3, Opción A",Razone la veracidad o falsedad de las siguient...,other


In [11]:
subset[(subset.year == 2024) & (subset.topic == 'Reacciones Redox')].statement

Series([], Name: statement, dtype: object)

In [12]:
subset.loc[1588, 'statement']

KeyError: 1588